In [1]:
import pandas as pd
import numpy as np
import talib
import pandas_profiling as pdp
import sweetviz as sv

import os
import sys
from pygam import s, LinearGAM
import category_encoders as ce
from typing import List, Dict

import matplotlib.pyplot as plt
import seaborn as sns

import ptitprince
from matplotlib_venn import venn2
plt.rcParams["font.family"] = "Hiragino Maru Gothic Pro"

from common_module.utils.plot import (
    plot_venn,
    plot_histogram,
    plot_raincloud,
    plot_heatmap,
    plot_histgram_for_valid,
    plot_lineargam,
    plot_feature_importance_for_valid
)
from common_module.utils.summarize import (
    get_pandas_profiling,
    get_sweetviz_report
)
from common_module.utils.preprocessor import (
    OrdinalEncodingBlock,
    CountEncodingBlock,
    GroupingBlock
)
from common_module.utils.format_df import format_df
from common_module.utils.utils import reduce_mem_usage

from probspace_usstock.modules.config_manager import ConfigManager
from probspace_usstock.modules.features import (
    create_company_features,
    calc_Symbol_lag_features,
    calc_Symbol_lag_log_features,
    # calc_List_lag_features,
    create_percentile_flg_columns,
    create_ymd_features
)
from probspace_usstock.modules.preprocessor import (
    melt_dataframe,
    create_base_dataframe
)
from probspace_usstock.modules.train_module import TrainLGBModule
from probspace_usstock.modules.predict import make_pred_df

sys.path.append("../")
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 100)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
# read
df_train_origin = pd.read_csv("../input/train_data.csv", encoding="utf-8-sig")
df_company_list_origin = pd.read_csv("../input/company_list.csv", encoding="utf-8-sig")
df_sub_template = pd.read_csv("../input/submission_template.csv", encoding="utf-8-sig")

# shape
display(df_train_origin.shape)
display(df_company_list_origin.shape)
display(df_sub_template.shape)

(420, 3279)

(7007, 6)

(3278, 2)

In [3]:
config_path = "../config/setting.yaml"
config_manager = ConfigManager(config_path=config_path)

In [4]:
# company関連の特徴量を先に作ってListをマージしたい
df_company_features = create_company_features(
    input=df_company_list_origin,
    duplicate_companies=config_manager.duplicate_companies
)
display(df_company_features.head(2))
display(df_company_features.shape)

,Symbol,IPOyear,Sector,List
0,XXII,NaN,Consumer Non-Durables,AMEX
1,FAX,1986.0,NaN,AMEX


(6991, 4)

In [5]:
df_base = melt_dataframe(input=df_train_origin)
df_base = create_base_dataframe(input=df_base, df_company=df_company_features)

display(df_base.head(2))

,Date,Symbol,List,stock_price
0,2011/11/13,VGSH,NASDAQ,55.942184
1,2011/11/20,VGSH,NASDAQ,55.978844


In [6]:
%%time

df_ymd = create_ymd_features(input=df_base)
df_Symbol_lag = calc_Symbol_lag_log_features(input=df_base)
# df_Symbol_lag = calc_Symbol_lag_features(input=df_base)
# df_List_lag = reduce_mem_usage(calc_List_lag_features(input=df_Symbol_lag))
df_flg = create_percentile_flg_columns(input=df_Symbol_lag)

display(df_ymd.head(2))
display(df_Symbol_lag.head(2))
# display(df_List_lag.head(2))
display(df_flg.head(2))

,Date,year,month,day,week_of_month,week_of_year
0,2011/11/13,2011,11,13,2,45
1,2011/11/20,2011,11,20,3,46


,Date,Symbol,stock_price_log1p,stock_price_log1p_1s,stock_price_log1p_2s,stock_price_log1p_3s,stock_price_log1p_4s,increase_stock_price_log1p_1s2s,increase_stock_price_log1p_2s3s,increase_stock_price_log1p_3s4s,stock_price_log1p_1s_4r_mean,stock_price_log1p_1s_4r_median,stock_price_log1p_1s_4r_max,stock_price_log1p_1s_4r_min,stock_price_log1p_1s_8r_mean,stock_price_log1p_1s_8r_median,stock_price_log1p_1s_8r_max,stock_price_log1p_1s_8r_min,stock_price_log1p_1s_12r_mean,stock_price_log1p_1s_12r_median,stock_price_log1p_1s_12r_max,stock_price_log1p_1s_12r_min
0,2011/11/13,VGSH,4.042036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011/11/20,VGSH,4.042680,4.042036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Date,Symbol,is_05_percentile_stock_price,is_95_percentile_stock_price
0,2011/11/13,VGSH,0,0
1,2011/11/20,VGSH,0,0


CPU times: user 1min 21s, sys: 1.79 s, total: 1min 23s
Wall time: 1min 34s


In [7]:
%%time

# _df_base = df_base[pd.to_datetime(df_base["Date"]) < pd.to_datetime("2013-01-01")].copy()

df_merge = pd.merge(df_base, df_company_features.drop("List", axis=1), how="left", on="Symbol")
df_merge = pd.merge(df_merge, df_ymd, how="left", on="Date")
df_merge = pd.merge(df_merge, df_Symbol_lag, how="left", on=["Date", "Symbol"])
# df = pd.merge(df, df_List_lag, how="left", on=["Date", "List"])
df_merge = pd.merge(df_merge, df_flg, how="left", on=["Date", "Symbol"])

CPU times: user 1.7 s, sys: 654 ms, total: 2.35 s
Wall time: 2.38 s


In [8]:
# NaNがある期間は学習に使用しない
print(df_merge.shape)
df_merge = df_merge[df_merge["stock_price_log1p_1s_12r_mean"].notnull()]
# df_merge = df_merge[df_merge["stock_price_1s_12r_mean"].notnull()]
print(df_merge.shape)
df_merge = reduce_mem_usage(df_merge)
df_merge.info()

(1376760, 33)
(1337424, 33)
Memory usage of dataframe is 346.93 MB
Memory usage after optimization is: 84.35 MB
Decreased by 75.7%
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337424 entries, 12 to 1376759
Data columns (total 33 columns):
 #   Column                           Non-Null Count    Dtype   
---  ------                           --------------    -----   
 0   Date                             1337424 non-null  category
 1   Symbol                           1337424 non-null  category
 2   List                             1335792 non-null  category
 3   stock_price                      1334146 non-null  float16 
 4   IPOyear                          441864 non-null   float16 
 5   Sector                           1114656 non-null  category
 6   year                             1337424 non-null  int16   
 7   month                            1337424 non-null  int8    
 8   day                              1337424 non-null  int8    
 9   week_of_month                    13

In [9]:
df_merge = format_df(df_merge, config_manager.feature_columns)

In [11]:
def split_dataframe(input: pd.DataFrame, config: Dict) -> pd.DataFrame:
    df_train = input[
        (pd.to_datetime(input["Date"]) >= pd.to_datetime(config.train_period[0])) &
        (pd.to_datetime(input["Date"]) <= pd.to_datetime(config.train_period[1]))
    ].copy()
    df_val = input[
        (pd.to_datetime(input["Date"]) >= pd.to_datetime(config.valid_period[0])) &
        (pd.to_datetime(input["Date"]) <= pd.to_datetime(config.valid_period[1]))
    ].copy()
    df_test = input[
        (pd.to_datetime(input["Date"]) >= pd.to_datetime(config.test_period[0])) &
        (pd.to_datetime(input["Date"]) <= pd.to_datetime(config.test_period[1]))
    ].copy()
    return df_train, df_val, df_test

df_train, df_val, df_test = split_dataframe(input=df_merge, config=config_manager)

In [12]:
def calc_lgb_weight(input: pd.DataFrame) -> pd.DataFrame:
    return 1 / input["stock_price_log1p_1s_4r_mean"].replace(0, 1) * 10

weight_train = calc_lgb_weight(df_train)
weight_val = calc_lgb_weight(df_val)

In [21]:
import lightgbm as lgb

y_train = df_train[config_manager.pred_cols].copy()
y_val = df_val[config_manager.pred_cols].copy()
X_train = df_train[config_manager.feature_columns.keys()].drop(config_manager.primary_key, axis=1).copy()
X_val = df_val[config_manager.feature_columns.keys()].drop(config_manager.primary_key, axis=1).copy()

train_data = lgb.Dataset(X_train, label=y_train, weight=weight_train)
val_data = lgb.Dataset(X_val, label=y_val, weight=weight_val)

## Optuna

In [22]:
# optuna経由でLightGBMをインポート
import optuna.integration.lightgbm as LGB_optuna

In [23]:
# 固定するパラメータ
fixed_params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "verbosity": -1
}

# Optunaでのパラメータ探索
model = LGB_optuna.train(
    params=fixed_params,
    train_set=train_data,
    valid_sets=val_data,
    verbose_eval=1000,
    early_stopping_rounds=100,
)

# 最適なパラメータの表示
best_params = model.params
print("Best params:", best_params)

[I 2022-03-18 18:01:22,302] A new study created in memory with name: no-name-15b874e3-3bdd-4ec4-af99-9093efa64eab


  0%|                                                                                                                           | 0/7 [00:00<?, ?it/s]

feature_fraction, val_score: inf:   0%|                                                                                         | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4225
[LightGBM] [Info] Number of data points in the train set: 1294810, number of used features: 28
[LightGBM] [Info] Start training from score 2.442752
Training until validation scores don't improve for 100 rounds




feature_fraction, val_score: 0.060737:   0%|                                                                                    | 0/7 [00:21<?, ?it/s]

feature_fraction, val_score: 0.060737:  14%|##########8                                                                 | 1/7 [00:21<02:07, 21.19s/it][I 2022-03-18 18:01:43,520] Trial 0 finished with value: 0.06073659305911132 and parameters: {'feature_fraction': 0.5}. Best is trial 0 with value: 0.06073659305911132.


feature_fraction, val_score: 0.060737:  14%|##########8                                                                 | 1/7 [00:21<02:07, 21.19s/it]

Early stopping, best iteration is:
[431]	valid_0's rmse: 0.0607366
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023072 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4225
[LightGBM] [Info] Number of data points in the train set: 1294810, number of used features: 28
[LightGBM] [Info] Start training from score 2.442752
Training until validation scores don't improve for 100 rounds




feature_fraction, val_score: 0.060737:  14%|##########8                                                                 | 1/7 [00:36<02:07, 21.19s/it]

feature_fraction, val_score: 0.060737:  29%|#####################7                                                      | 2/7 [00:36<01:27, 17.46s/it][I 2022-03-18 18:01:58,359] Trial 1 finished with value: 0.06116450751246555 and parameters: {'feature_fraction': 0.6}. Best is trial 0 with value: 0.06073659305911132.


feature_fraction, val_score: 0.060737:  29%|#####################7                                                      | 2/7 [00:36<01:27, 17.46s/it]

...

Early stopping, best iteration is:
[201]	valid_0's rmse: 0.0609705
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.040089 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4225
[LightGBM] [Info] Number of data points in the train set: 1294810, number of used features: 28
[LightGBM] [Info] Start training from score 2.442752
Training until validation scores don't improve for 100 rounds




min_data_in_leaf, val_score: 0.060542:  60%|#############################################6                              | 3/5 [01:04<00:33, 16.77s/it]

min_data_in_leaf, val_score: 0.060542:  80%|############################################################8               | 4/5 [01:04<00:15, 15.49s/it][I 2022-03-18 18:24:05,641] Trial 66 finished with value: 0.061078779847547005 and parameters: {'min_child_samples': 10}. Best is trial 63 with value: 0.060808402880029415.


min_data_in_leaf, val_score: 0.060542:  80%|############################################################8               | 4/5 [01:04<00:15, 15.49s/it]

Early stopping, best iteration is:
[168]	valid_0's rmse: 0.0610788
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034605 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4225
[LightGBM] [Info] Number of data points in the train set: 1294810, number of used features: 28
[LightGBM] [Info] Start training from score 2.442752
Training until validation scores don't improve for 100 rounds




min_data_in_leaf, val_score: 0.060542:  80%|############################################################8               | 4/5 [01:17<00:15, 15.49s/it]

min_data_in_leaf, val_score: 0.060542: 100%|############################################################################| 5/5 [01:17<00:00, 15.47s/it]

Early stopping, best iteration is:
[156]	valid_0's rmse: 0.0610192
Best params: {'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'rmse', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 31, 'feature_fraction': 0.8, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100, 'categorical_column': [0, 2, 3, 4, 5, 6]}


In [24]:
# 最適なパラメータの表示
best_params = model.params
print("Best params:", best_params)

Best params: {'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'rmse', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 31, 'feature_fraction': 0.8, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100, 'categorical_column': [0, 2, 3, 4, 5, 6]}
